In [17]:
import arxiv
import urllib
import pdfx
import re

# Téléchargement des PDF sur Arkiv

In [10]:
#Utilisation de la librairie ArXiv pou récupérer le lien des pdf

search = arxiv.Search(
    query = "computer science &  ai",
    # id_list=["1605.08386v1"]
    max_results = 1,
    sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in search.results():
    print ("Titre :", result.title)
    print ("Auteurs :", result.authors)
    print ("Lien :", result.pdf_url)

Titre : MaxViT: Multi-Axis Vision Transformer
Auteurs : [arxiv.Result.Author('Zhengzhong Tu'), arxiv.Result.Author('Hossein Talebi'), arxiv.Result.Author('Han Zhang'), arxiv.Result.Author('Feng Yang'), arxiv.Result.Author('Peyman Milanfar'), arxiv.Result.Author('Alan Bovik'), arxiv.Result.Author('Yinxiao Li')]
Lien : http://arxiv.org/pdf/2204.01697v1


In [27]:
# On met les auteurs sur forme de liste pour faciliter le traitement

for result in search.results():
    author = result.authors
    author_list = [re.sub("[^A-Za-z0-9]","_",str(i)) for i in author]

print (author_list)

['Zhengzhong_Tu', 'Hossein_Talebi', 'Han_Zhang', 'Feng_Yang', 'Peyman_Milanfar', 'Alan_Bovik', 'Yinxiao_Li']


In [11]:
# Download the pdf & store it

response = urllib.request.urlopen(result.pdf_url)    
file = open("../data/Test" + ".pdf", 'wb')
file.write(response.read())
file.close()

In [12]:
# Leture du PDF

pdf = pdfx.PDFx("../data/Test.pdf")
metadata = pdf.get_metadata()
references_list = pdf.get_references()
references_dict = pdf.get_references_as_dict()
text = pdf.get_text()

In [31]:
print (metadata)
#print (references_list)
print (references_dict['url'][0])
#print (text)

{'CreationDate': 'D:20220405011353Z', 'Creator': 'LaTeX with hyperref', 'ModDate': 'D:20220405011353Z', 'PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'Producer': 'pdfTeX-1.40.21', 'Trapped': 'False', 'Pages': 28}
2107.00641


## Optimisation du champ de recherche à partir du mot "Références"

In [14]:
def after_references(mypdftext): 
    keyword1 = 'References'
    keyword2 = 'REFERENCES'
    keyword3 = 'R EFERENCES'
    keyword4 = 'Reference'
    keyword5='[1]' 

    if keyword1 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword1)
    elif keyword2 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword2)
    elif keyword3 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword3)
    elif keyword4 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword4)
    elif keyword5 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword5)
    else:
        after_keyword = mypdftext[:10000]
    return after_keyword

#All references in a variable

references=after_references(text)
# print(references)

# Recherche des entités nommées

In [15]:
import spacy
from spacy.lang.fr.examples import sentences 

def extract(text:str) :
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text.strip())
    named_entities = []
    
    for i in doc.ents:
        entry = str(i.lemma_).lower()
        text = text.replace(str(i).lower(), "")
        if i.label_ in ["PERSON"]:
            named_entities.append(entry.title().replace(" ", "_").replace("\n","_"))
        named_entities = list(dict.fromkeys(named_entities))
    return named_entities

print (extract(references)[4])

Zhou


# Création de l'ontologie

In [ ]:
from owlready2 import *

In [ ]:
owlready2.JAVA_EXE='/usr/bin/java'

In [ ]:
onto = get_ontology('http://filrouge.org/') # nouvelle ontologie, on donne son IRI

with onto:
    # Define an Author
    class Author(Thing):  # nouvelle classe sous-classe de Thing
        pass
    
    # Define a Reference
    class Reference(Thing): pass
    
    # An Author get ideas from theirs references
    class getIdeasFrom(Author >> Reference): pass
        
    # An Author is influenced by the references of the references
    class isInfluencedBy(Author >> Reference): pass
